In [13]:
import random

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, log_loss, roc_auc_score

from src.dataset import get_dataset
from src.models.dualemb import DualEmbPredictor
from src.models.elog import ELOgPredictor
from src.models.freq import FrequencyMatchPredictor
from src.models.uni import UniformMatchPredictor

In [14]:
ITERATIONS = 1
random.seed(5438)
np.random.seed(5438)

In [15]:
def determine_target(row):
    if row["team_score"] > row["opponent_score"]:
        return 0
    elif row["team_score"] == row["opponent_score"]:
        return 1
    else:
        return 2

In [16]:
dataset = get_dataset()

/mnt/c/Personal/football-matches-prediction/src/dataset.py:47: DtypeWarning: Columns (11,14,50,102,123,125,129,131,133,135,137,139,141,143,145,147,149,157,158,160,161,163,165,167,169,171,173,184,186,187,205,206) have mixed types. Specify dtype option on import or set low_memory=False.
  mls_df = pd.read_csv("data/mls_matches.csv")


In [17]:
dataset

,team_id,opponent_id,team_at_home,opponent_at_home,team_score,opponent_score,fold
0,Scotland,England,1.0,0.0,0,0,international
1,England,Scotland,1.0,0.0,4,2,international
2,Scotland,England,1.0,0.0,2,1,international
3,England,Scotland,1.0,0.0,2,2,international
4,Scotland,England,1.0,0.0,3,0,international
...,...,...,...,...,...,...,...
143000,Sport Lisboa e Benfica,Sporting Clube de Braga,1.0,0.0,3,0,europe
143001,Panathinaikos Athlitikos Omilos,APS Atromitos Athinon,1.0,0.0,2,1,europe
143002,Fulham Football Club,Watford FC,1.0,0.0,4,1,europe
143003,Panthessalonikios Athlitikos Omilos Konstantin...,Athlitiki Enosi Konstantinoupoleos,1.0,0.0,1,1,europe


In [18]:
model_classes = [
    # FrequencyMatchPredictor,
    # UniformMatchPredictor,
    # ELOgPredictor,
    DualEmbPredictor,
]
# folds_names = ["brazil", "libertadores", "mls", "europe", "international"]
folds_names = ["brazil", "international"]

In [19]:
folds_train = [dataset[dataset["fold"] != name] for name in folds_names]
folds_test = [dataset[dataset["fold"] == name] for name in folds_names]

In [20]:
results = pd.DataFrame({}, columns=["metric", "model", "fold", "iteration", "value"])

In [21]:
for iteration in range(ITERATIONS):
    for model_class in model_classes:
        for fold_train, fold_test, fold_test_name in zip(
            folds_train, folds_test, folds_names
        ):
            X_train = fold_train[
                ["team_id", "opponent_id", "team_at_home", "opponent_at_home"]
            ]
            y_train = fold_train[["team_score", "opponent_score"]]
            X_test = fold_test[
                ["team_id", "opponent_id", "team_at_home", "opponent_at_home"]
            ]
            y_test = fold_test[["team_score", "opponent_score"]]
            model = model_class()
            model.fit(X_train, y_train)
            pred = model.predict_and_update(X_test, y_test)
            max_pred = np.argmax(pred, axis=1)
            target = fold_test.apply(determine_target, axis=1).to_numpy()
            report = classification_report(
                target, max_pred, target_names=["win", "draw", "loss"], output_dict=True
            )
            metrics = {
                "accuracy": report["accuracy"],
                "log_loss": log_loss(target, pred, labels=[0, 1, 2]),
                "micro_auc_roc": roc_auc_score(
                    target, pred, average="micro", multi_class="ovr"
                ),
                "weighted_precision": report["weighted avg"]["precision"],
                "weighted_recall": report["weighted avg"]["recall"],
                "macro_precision": report["macro avg"]["precision"],
                "macro_recall": report["macro avg"]["recall"],
            }
            for key, value in metrics.items():
                results.loc[len(results)] = {
                    "metric": key,
                    "model": model_class.__name__,
                    "fold": fold_test_name,
                    "iteration": iteration + 1,
                    "value": value,
                }

Epoch 1/100


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3699/3699 [00:06<00:00, 557.83it/s, loss=1.1740]


Epoch 2/100


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3699/3699 [00:06<00:00, 529.73it/s, loss=1.1442]


Epoch 3/100


...

Epoch 99/100


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3699/3699 [00:06<00:00, 528.92it/s, loss=1.1424]


Epoch 100/100


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3699/3699 [00:09<00:00, 407.29it/s, loss=1.1417]
/mnt/c/Personal/football-matches-prediction/src/models/dualemb.py:307: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pair_X["team_id"] = [0, 1]
/mnt/c/Personal/football-matches-prediction/src/models/dualemb.py:308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pair_X["opponent_id"] = [1, 0]
/home/raze

ValueError: Input contains NaN.

In [25]:
pred[:20]

array([[4.26255000e-01, 2.73424798e-01, 3.00320202e-01],
       [4.26255000e-01, 2.73424798e-01, 3.00320202e-01],
       [4.26255000e-01, 2.73424798e-01, 3.00320202e-01],
       [4.26255000e-01, 2.73424798e-01, 3.00320202e-01],
       [4.26255000e-01, 2.73424798e-01, 3.00320202e-01],
       [4.26255000e-01, 2.73424798e-01, 3.00320202e-01],
       [4.26255000e-01, 2.73424798e-01, 3.00320202e-01],
       [4.26255000e-01, 2.73424798e-01, 3.00320202e-01],
       [4.26255000e-01, 2.73424798e-01, 3.00320202e-01],
       [4.26255000e-01, 2.73424798e-01, 3.00320202e-01],
       [9.75563858e-01, 1.65113096e-02, 7.92483276e-03],
       [3.61708515e-01, 2.78201390e-01, 3.60090094e-01],
       [9.47484711e-01, 3.51405642e-02, 1.73747249e-02],
       [3.53221984e-01, 2.78133031e-01, 3.68644985e-01],
       [5.09393649e-02, 9.30987631e-02, 8.55961872e-01],
       [9.78512027e-01, 1.45329812e-02, 6.95499186e-03],
       [4.69978128e-01, 2.65520926e-01, 2.64500946e-01],
       [1.14814079e-06, 2.45239

In [ ]:
results

In [ ]:
results.groupby(["metric", "model", "fold"])["value"].mean().reset_index().groupby(
    ["metric", "model"]
)["value"].mean().reset_index().pivot(index="model", columns="metric", values="value")